In [110]:
import pandas as pd
from apyori import apriori
import plotly.express as px

data = pd.read_csv('games.csv', sep=',', low_memory=False)
data1 = data.iloc[:, [2,3,7,9,10,11,12]]
data1['Winner'] = data1.apply(lambda row: row['Home'] 
                              if row['HomeGoals'] > row['AwayGoals'] 
                              else (row['Away'] if row['AwayGoals'] > row['HomeGoals'] else 'Draw'), axis=1)
data1 = data1.head(7500)
# data2 = data1[['Season_End_Year', 'Winner']]
data1.head()

C:\Users\opawk\AppData\Local\Temp\ipykernel_19532\4182781069.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Season_End_Year,Date,Home,Away,HomeGoals,AwayGoals,Winner
0,GER,2015,8/22/2014,Bayern Munich,Wolfsburg,2,1,Bayern Munich
1,GER,2015,8/23/2014,Eint Frankfurt,Freiburg,1,0,Eint Frankfurt
2,GER,2015,8/23/2014,Hannover 96,Schalke 04,2,1,Hannover 96
3,GER,2015,8/23/2014,Hertha BSC,Werder Bremen,2,2,Draw
4,GER,2015,8/23/2014,Hoffenheim,Augsburg,2,0,Hoffenheim


In [111]:
# Включаем команду и год в каждую транзакцию
data1['Transaction'] = data1.apply(lambda row: [row['Country'], str(row['Season_End_Year']), row['Winner']], axis=1)
data1['Transaction1'] = data1.apply(lambda row: [row['Home'], row['Winner']],  axis=1)
# Выбираем только необходимые столбцы
data2 = data1[['Transaction']]
data3 = data1[['Transaction1']]
# Преобразуем результат в список списков
transactions = data2['Transaction'].tolist()
transactions1 = data3['Transaction1'].tolist()
# Задаем параметры для алгоритма apriori
min_support = 0.001
min_confidence = 0.001
min_lift = 1.1
min_length = 2
max_length = 2

# Запускаем алгоритм apriori
rules = apriori(transactions=transactions, min_support=min_support, min_confidence=min_confidence, min_lift=min_lift, min_length=min_length, max_length=max_length)
rules1 = apriori(transactions=transactions1, min_support=min_support, min_confidence=min_confidence, min_lift=min_lift, min_length=min_length, max_length=max_length)

# Преобразуем результаты в список
result = list(rules)
result1 = list(rules1)

# Выводим первую транзакцию и первый результат
print(transactions[0])
print(result[0])
print(transactions1[0])
print(result1[0])

['GER', '2015', 'Bayern Munich']
RelationRecord(items=frozenset({'Almería', '2015'}), support=0.0010666666666666667, ordered_statistics=[OrderedStatistic(items_base=frozenset({'2015'}), items_add=frozenset({'Almería'}), confidence=0.0075046904315197, lift=7.0356472795497185), OrderedStatistic(items_base=frozenset({'Almería'}), items_add=frozenset({'2015'}), confidence=1.0, lift=7.0356472795497185)])
['Bayern Munich', 'Bayern Munich']
RelationRecord(items=frozenset({'Draw', 'Arminia'}), support=0.0016, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Arminia'}), items_add=frozenset({'Draw'}), confidence=0.30000000000000004, lift=1.1987213638785297), OrderedStatistic(items_base=frozenset({'Draw'}), items_add=frozenset({'Arminia'}), confidence=0.006393180607352157, lift=1.1987213638785295)])


In [112]:
def inspect(results):
    lhs = [tuple(result[2][0][0])[0] for result in results]
    rhs = [tuple(result[2][0][1])[0] for result in results]
    supports = [result[1] for result in results]
    confidence = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]
    return list (zip(lhs,rhs,supports,confidence,lifts))

resultsinDataFrame = pd.DataFrame(inspect(result), columns=["Left hand side", "Right hand side", "Support", "Confidence", "lifts"])

resultsinDataFrame = resultsinDataFrame.nlargest(n=10,columns="lifts")

resultsinDataFrame

# resultsinDataFrame1 = pd.DataFrame(inspect(result1), columns=["Left hand side", "Right hand side", "Support", "Confidence", "lifts"])

# resultsinDataFrame1 = resultsinDataFrame1.nlargest(n=10,columns="lifts")

# resultsinDataFrame1

# Left hand side (LHS) - это предшествующий элемент или элементы, на которые существенно влияет предлагаемый элемент или элементы.

# Right hand side (RHS) - это предлагаемый элемент или элементы, которые существенно влияют на предшествующий элемент или элементы.

# Support - это вероятность того, что правило встретится в данной транзакции. В нашем случае, значение поддержки меньше 0.001, поэтому это значительное правило.

# Confidence - это показатель вероятности того, что правило будет действовать только при условии, что LHS также существует.

# Lifts - это показатель, который определяет, насколько LHS и RHS связаны. Значение lifts выше 1 указывает на увеличение соотношения.

,Left hand side,Right hand side,Support,Confidence,lifts
218,2022,Bochum,0.001600,0.017493,10.932945
0,2015,Almería,0.001067,0.007505,7.035647
17,2015,QPR,0.001067,0.007505,7.035647
125,2019,Cardiff City,0.001333,0.009381,7.035647
180,2021,Arminia,0.001200,0.013120,7.028322
46,2016,Norwich City,0.001200,0.008443,6.332083
187,2021,Cádiz,0.001467,0.016035,6.329600
235,2022,Mallorca,0.001333,0.014577,5.754181
148,2019,Wolves,0.002133,0.015009,5.360493
107,2018,Huddersfield,0.001200,0.008443,5.276735


In [100]:
import plotly.express as px

data_10_best = resultsinDataFrame[:10]
fig = px.scatter(data_10_best, x = 'Support', y = 'Confidence', color='lifts')
fig1 = px.scatter(data_10_best, x = 'Left hand side', y = 'Right hand side', color='lifts', size='Support')
fig.show()
fig1.show()

In [68]:
print("10 лучших команд по кол-ву побед")
x = data1['Winner'].value_counts().sort_values(ascending=False)[:10]
fig = px.bar(x= x.index, y= x.values)
fig.update_layout(title_text= "10 лучших команд по кол-ву побед", xaxis_title= "Команды", yaxis_title="Кол-во")
fig.show()

10 лучших команд по кол-ву побед


In [84]:
data1 = data1.astype(str)
transactions = data1.values.tolist()
# Задаем параметры для алгоритма apriori
min_support = 0.001
min_confidence = 0.001
min_lift = 1.1
min_length = 2
max_length = 2

# Запускаем алгоритм apriori
rules = apriori(transactions=transactions, min_support=min_support, min_confidence=min_confidence, min_lift=min_lift, min_length=min_length, max_length=max_length)

# Преобразуем результаты в список
result = list(rules)

print(transactions[0])
print(result[0])

['GER', '2015', '8/22/2014', 'Bayern Munich', 'Wolfsburg', '2', '1', 'Bayern Munich']
RelationRecord(items=frozenset({'1/13/2018', '0'}), support=0.0012, ordered_statistics=[OrderedStatistic(items_base=frozenset({'0'}), items_add=frozenset({'1/13/2018'}), confidence=0.0025231286795626574, lift=1.1131450056894077), OrderedStatistic(items_base=frozenset({'1/13/2018'}), items_add=frozenset({'0'}), confidence=0.5294117647058822, lift=1.1131450056894074)])
